In [2]:
#Step 1: import necessary packages
import pandas as pd

#Step 2: visualise expense list
dataset = pd.read_excel('expenses_splitter_example_file.xlsx')
pd.set_option('display.width', 200) #(line below as well) used to display all columns in one set
pd.set_option('display.max_columns',8) 
print(dataset)

                     expense name       Date   cost Paid by  Lily  Bob  Emma  Anna
0                     cooking oil 2024-07-04   1.70    Emma     1    1     1     1
1                 concert tickets 2024-08-08  24.00    Emma     0    0     0     1
2                     settle debt 2024-08-09  24.85    Anna     0    0     1     0
3                       shoe rack 2024-09-07  18.00     Bob     0    1     1     1
4           ikea kitchen supplies 2024-09-07  17.35    Emma     1    1     1     1
5           toilet paper and soap 2024-09-10   7.00    Lily     1    1     1     1
6                    house dinner 2024-09-16  20.00     Bob     1    1     1     1
7                        kruidvat 2024-09-16   2.50    Lily     0    1     0     0
8                    gyoza dinner 2024-09-22  17.60    Lily     1    1     1     1
9                ikea shower rack 2024-09-30  15.00    Emma     0    1     1     1
10      higher gas prices october 1900-01-04  13.21    Lily     1    1     1     1
11  

In [1]:
#import necessary packages. Importing again so that if user does not wish to see expense file the code can still run
import pandas as pd

#Step 2: create date filtering function
def filter_expenses_by_date(expenses, start_date, end_date):
    split_expenses_dataset = pd.read_excel(expenses)
    
    #this filters the dataset based on a given start and end date
    filtered_dataset = split_expenses_dataset[(split_expenses_dataset['Date'] >= start_date) & (split_expenses_dataset['Date'] <= end_date)]
    
    return filtered_dataset
    

#Step 3: defining function that calculates the expenses
def split_expenses(expenses, start_date = None, end_date = None):
    if start_date and end_date: #we need an "if" statement so that if there are dates given for filtering the data it uses the previous filter function
        split_expenses_dataset = filter_expenses_by_date(expenses, start_date, end_date)
    else: #if there are no start and end dates provided, then the code will run the expense calculation on the entire dataset
        split_expenses_dataset = pd.read_excel(expenses)
        
    #Step 4: define payer and participant columns outside of loop
    payer_column = split_expenses_dataset.columns[3] 
    participant_columns = split_expenses_dataset.columns[4:] 

    #Step 5: create dictionary that keeps track of expenses
    balances = {}

    #Step 6: create a loop that calculates the expenses for each row and adds them all up 
    for _, row in split_expenses_dataset.iterrows():
        #Step 6b: define the content of the rows found in the dataset
        cost = row["cost"]
        payer = row["Paid by"]

        #Step 7: this reads who participated in an expense
        #line below: extracts the value from the row that corresponds to the participant_columns, previously defined as columns 5+ in the excel
        participants = row[participant_columns]
        #line below: we only select the participants (as defined above) so we don't run through all columns. 
        #Then it filters through to the columns where the value is 1 (e.g. the person participated in the expense)
        #Then the indextolist reads the names of the participating people out of the top row
        involved_people = participants[participants == 1].index.tolist()

        #Step 8: define how the costs get shared
        number_involved_people = len(involved_people) #counts the amount of people who contributed to the expense
        cost_share = cost/number_involved_people 
            
        #Step 9: add total expense cost to the balance of the payer
        balances[payer] = balances.get(payer, 0) + cost 

        #Step 10: subtract expense cost based on the cost share for participants in an expense
        for participant in involved_people:
            balances[participant] = balances.get(participant, 0) - cost_share

        
    #Step 11: finish the function
    return balances

#Step 12: calculate split expenses and display final amounts. Here, change the file name, start and end dates if using a different dataset.
file_name = "expenses_splitter_example_file.xlsx"
start_date = None
end_date = None
final_balance = split_expenses(file_name, start_date, end_date)
print(final_balance)

{'Emma': -3.345833333333334, 'Lily': 75.1125, 'Bob': -24.37083333333334, 'Anna': -47.39583333333334}
